In [1]:
from dotenv import load_dotenv

MODEL = "llama2"

In [3]:
from langchain_ollama import OllamaLLM,OllamaEmbeddings 
model = OllamaLLM(model=MODEL)
model.invoke("Tell me a joke")
embaddings = OllamaEmbeddings(model=MODEL)

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

"Sure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope that made you smile!"

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("ADVIL.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'producer': 'www.adlibsoftware.com: CTP (5.10.0.7216) OS (Windows 2012,2,0,64)', 'creator': 'Microsoft Word(12.0)', 'creationdate': '2016-09-19T17:38:26-04:00', 'author': 'mazuren', 'moddate': '2021-09-29T13:31:53+05:30', 'r_object_id': '090177e189b0de81', 'source': 'ADVIL.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Advil Tablets\nIbuprofen Tablets, 200 mg\nThis is the most current labeling information and may differ from labels on product packaging. If there are any \ndifferences between this website labeling and product packaging labeling, this website labeling should be regarded \nas the most current.\nLBL-00000027 (Version 38.0) Page 1 of 3\nDrug Facts\nActive ingredient (in each tablet)                                           Purpose\nIbuprofen 200 mg (NSAID)*   ………………………………….Pain reliever/Fever reducer\n*nonsteroidal anti-inflammatory drug\nUses\n\uf06e temporarily relieves minor aches and pains due to:\n\uf06e headache \n\uf06e toot

In [6]:
from langchain.prompts import PromptTemplate

template = """
        Answer the question based on the context below. If you can't
        answer the question, reply "I don't know".

        Context: {context}

        Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


        Answer the question based on the context below. If you can't
        answer the question, reply "I don't know".

        Context: Here is some context

        Question: Here is a question



In [7]:
chain = prompt | model | parser 

In [8]:
chain.input_schema.model_json_schema()

{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [8]:
chain.invoke(
    {
        "context": "The name I was given was Denisa",
        "question": "What's my name?"
    }
)

' Hello Denisa! Your name is Denisa.'

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embaddings)

c:\Users\dunca\OneDrive\Desktop\Disertation\LLM\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
retriever = vectorstore.as_retriever()

retriever.invoke("ADVIL")

[Document(metadata={'source': 'ADVIL.pdf', 'page': 2}, page_content='Advil Tablets\nIbuprofen Tablets, 200 mg\nThis is the most current labeling information and may differ from labels on product packaging. If there are any \ndifferences between this website labeling and product packaging labeling, this website labeling should be regarded \nas the most current.\nLBL-00000027 (Version 38.0) Page 3 of 3\nIf pregnant or breast-feeding, ask a health professional before use. It is especially \nimportant not to use ibuprofen at 20 weeks or later in pregnancy unless definitely \ndirected to do so by a doctor because it may cause problems in the unborn child or \ncomplications during delivery.\nKeep out of reach of children.  In case of overdose, get medical help or contact a \nPoison Control Center right away.\nDirections\n\uf06e do not take more than directed\n\uf06e the smallest effective dose should be used\n\uf06e adults and children 12 years and over: take 1 tablet every 4 to 6 hours whil

In [11]:
from operator import itemgetter

chain =(

{
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question")
} |

prompt | model | parser )

chain.invoke({
    "question":"What is advil?"
})

' Based on the context provided, Advil appears to be a brand name for a medication containing ibuprofen, which is a nonsteroidal anti-inflammatory drug (NSAID). The medication is used to temporarily relieve minor aches and pains, such as headache, toothache, backache, menstrual cramps, the common cold, muscular pain, and fever. It is important to follow the dosage instructions and precautions provided in the labeling information to avoid potential side effects or complications.'

In [12]:
questions = [
    "Can children under 12 years take advil?"
]

In [13]:
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question':question})}")
    print()

Question: Can children under 12 years take advil?
Answer: The answer to your question is: Ask a doctor. According to the labeling information provided, if you are unsure about whether children under 12 years can take Advil, you should "ask a doctor." It is important to consult with a healthcare professional before giving any medication, including Advil, to children under 12 years of age.
